In [17]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [18]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [19]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       1.00      0.88      0.94        34
           1       0.93      1.00      0.96        53

    accuracy                           0.95        87
   macro avg       0.96      0.94      0.95        87
weighted avg       0.96      0.95      0.95        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [20]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.70      0.82        43
           1       0.77      1.00      0.87        44

    accuracy                           0.85        87
   macro avg       0.89      0.85      0.85        87
weighted avg       0.88      0.85      0.85        87



# Lets add data, fakenews!

In [21]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [22]:
data = pd.concat([fakenews, df], ignore_index=True)


In [23]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.88      0.94        34
           1       0.76      0.98      0.86        49
           2       0.98      0.80      0.88        54

    accuracy                           0.88       137
   macro avg       0.91      0.89      0.89       137
weighted avg       0.91      0.88      0.89       137



In [24]:
confusion_matrix(y_test, y_pred)

array([[30,  4,  0],
       [ 0, 48,  1],
       [ 0, 11, 43]], dtype=int64)

## Is the vectorizer that good?

In [25]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [26]:
data = pd.concat([bbc, data], ignore_index=True)

In [27]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [28]:
df_shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93        21
           1       0.91      0.88      0.90        34
           2       0.90      0.90      0.90        29
           3       0.96      0.90      0.93        30
           4       0.97      0.91      0.94        33
           5       0.88      0.95      0.91        37
           6       0.90      0.93      0.92        29
           7       0.91      1.00      0.95        20

    accuracy                           0.92       233
   macro avg       0.92      0.92      0.92       233
weighted avg       0.92      0.92      0.92       233



In [29]:
confusion_matrix(y_test, y_pred)

array([[19,  2,  0,  0,  0,  0,  0,  0],
       [ 0, 30,  1,  0,  0,  0,  3,  0],
       [ 0,  1, 26,  0,  0,  2,  0,  0],
       [ 0,  0,  0, 27,  0,  2,  0,  1],
       [ 1,  0,  1,  0, 30,  0,  0,  1],
       [ 0,  0,  1,  1,  0, 35,  0,  0],
       [ 0,  0,  0,  0,  1,  1, 27,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 20]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.

## Lets create a transformer

In [30]:
df_deeplearning = df_shuffled

In [31]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Use tokenizer to encode
df_deeplearning['input_ids'] = df_deeplearning['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

In [32]:
df_deeplearning

,cleaned_text,label,input_ids
0,Dogged Federer claims Dubai crown World numbe...,7,"[101, 28844, 2098, 28294, 4447, 11558, 4410, 2..."
1,,2,"[101, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Be the First to Comment! Search articles,2,"[101, 2022, 1996, 2034, 2000, 7615, 999, 3945,..."
3,Clarke faces ID cards rebellion Charles Clark...,5,"[101, 8359, 5344, 8909, 5329, 7417, 2798, 8359..."
4,GB quartet get cross country call Four Britis...,7,"[101, 16351, 8530, 2131, 2892, 2406, 2655, 217..."
...,...,...,...
769,House prices show slight increase Prices of h...,3,"[101, 2160, 7597, 2265, 7263, 3623, 7597, 1997..."
770,Smith loses US box office crown New comedy Di...,6,"[101, 3044, 12386, 2149, 3482, 2436, 4410, 204..."
771,Tories plan migrant health checks Non-Europea...,5,"[101, 23413, 2229, 2933, 20731, 2740, 14148, 2..."
772,ITunes user sues Apple over iPod A user of Ap...,4,"[101, 11943, 5310, 9790, 2015, 6207, 2058, 263..."


In [33]:
input_ids = torch.tensor(df_deeplearning['input_ids'].tolist(), dtype=torch.long)
labels = torch.tensor(df_deeplearning['label'].tolist(), dtype=torch.float)  # Assuming multi-label

# Create torch dataset
dataset = TensorDataset(input_ids, labels)

train_length=int(0.7* len(dataset))

test_length=len(dataset)-train_length

train_dataset, test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length),
                                                          generator=torch.Generator().manual_seed(42))


train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [34]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)  
        return x
    

model_0 = SimpleModel(input_size=128, hidden_units=16, output_size=8)

In [35]:
input_ids.size()

torch.Size([774, 128])

In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr = 0.01)

In [37]:
def training_loop(loss_fn, optimizer, model,epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        train_loss = 0
        for batch, (X, y) in enumerate(train_dataloader):
            model.train()
            

            X, y = X.to(device), y.to(device)

            X = X.float() 
            y = y.long()  
            
            y_pred = model(X)

            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

        
        train_loss /= len(train_dataloader)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {train_loss:.4f}")


# Call the training loop
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_0, epochs=100)


Epoch 1/100, Average Training Loss: 181.0598
Epoch 21/100, Average Training Loss: 2.0774
Epoch 41/100, Average Training Loss: 2.0778
Epoch 61/100, Average Training Loss: 2.0766
Epoch 81/100, Average Training Loss: 2.0783


In [44]:
class MoreAdvancedModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units),
                                    nn.Dropout(0.3))
        self.layer2 = nn.Linear(hidden_units, hidden_units)
        self.layer3 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)  
        return x
    

model_1 = MoreAdvancedModel(input_size=128, hidden_units=512, output_size=8)

In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr = 0.01)

In [46]:
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_1, epochs=100)

Epoch 1/100, Average Training Loss: 1532.5715
Epoch 21/100, Average Training Loss: 2.0738
Epoch 41/100, Average Training Loss: 2.0746
Epoch 61/100, Average Training Loss: 2.0726
Epoch 81/100, Average Training Loss: 2.0726


In [47]:
all_preds = []
true_preds = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
model_1.eval()
with torch.no_grad():
    for data in test_loader:
        inputs = data[0].to(device).float()  
        outputs = model_1(inputs)
        preds = torch.argmax(outputs, dim=1)  # or outputs if you want raw logits/probs
        all_preds.append(preds)
        true_preds.append(data[1])

true_preds = torch.cat(true_preds).tolist()
final_preds = torch.cat(all_preds).tolist()

In [48]:
from collections import defaultdict, Counter

# Group predictions by true label
label_predictions = defaultdict(list)

for pred, true in zip(final_preds, true_preds):
    label_predictions[true].append(pred)

# Now sort the predictions by frequency for each true label
sorted_preds_per_class = {
    label: Counter(preds).most_common()  # Returns list of (pred, count) sorted by count
    for label, preds in label_predictions.items()
}


In [49]:
sorted_preds_per_class

{2.0: [(2, 30)],
 6.0: [(2, 35)],
 5.0: [(2, 30)],
 4.0: [(2, 27)],
 3.0: [(2, 28)],
 7.0: [(2, 31)],
 0.0: [(2, 24)],
 1.0: [(2, 28)]}

Just guesses 2 :/